In [1]:
#import libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

TO DO:
- Combine first and last name
- Create total minutes
- Rename division to gender
- Need to put city state in the lat lon

In [3]:
#!pip install geopy

In [4]:
# translate city/states to lat and log
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geopy.extra.rate_limiter import RateLimiter

In [6]:
df = pd.read_csv(r"ff_race_50.csv")
df.head()

,Place,First,Last,City,State,Age,Division,Time,Unnamed: 8,Unnamed: 9
0,1,Daniel,Wilson,Tulsa,OK,35,M,8:23:01,NaN,NaN
1,2,Eric,Davis,Greenwood,IN,38,M,8:57:54,NaN,8:21:45
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,9:24:35,NaN,NaN
3,4,Ron,Hammett,Montverde,FL,53,M,9:24:36,NaN,NaN
4,5,Seth,Cain,Geneva,FL,44,M,9:42:17,NaN,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Place       108 non-null    int64  
 1   First       108 non-null    object 
 2   Last        108 non-null    object 
 3   City        108 non-null    object 
 4   State       108 non-null    object 
 5   Age         108 non-null    int64  
 6   Division    108 non-null    object 
 7   Time        108 non-null    object 
 8   Unnamed: 8  0 non-null      float64
 9   Unnamed: 9  1 non-null      object 
dtypes: float64(1), int64(2), object(7)
memory usage: 8.6+ KB


In [8]:
df2 = df.dropna(axis=1)
df2.head()

,Place,First,Last,City,State,Age,Division,Time
0,1,Daniel,Wilson,Tulsa,OK,35,M,8:23:01
1,2,Eric,Davis,Greenwood,IN,38,M,8:57:54
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,9:24:35
3,4,Ron,Hammett,Montverde,FL,53,M,9:24:36
4,5,Seth,Cain,Geneva,FL,44,M,9:42:17


In [9]:
df2['fullname']=df2['First'] + ' ' + df2['Last']

df2.head()

<ipython-input-9-3f6825206b91>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['fullname']=df2['First'] + ' ' + df2['Last']


,Place,First,Last,City,State,Age,Division,Time,fullname
0,1,Daniel,Wilson,Tulsa,OK,35,M,8:23:01,Daniel Wilson
1,2,Eric,Davis,Greenwood,IN,38,M,8:57:54,Eric Davis
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,9:24:35,Stewart Edwards
3,4,Ron,Hammett,Montverde,FL,53,M,9:24:36,Ron Hammett
4,5,Seth,Cain,Geneva,FL,44,M,9:42:17,Seth Cain


In [10]:
df2['Time'] = pd.to_timedelta(df2['Time'])
df2.dtypes

<ipython-input-10-34cbe420ec79>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Time'] = pd.to_timedelta(df2['Time'])


Place                 int64
First                object
Last                 object
City                 object
State                object
Age                   int64
Division             object
Time        timedelta64[ns]
fullname             object
dtype: object

In [11]:
df2['Time'].dt.total_seconds()/ 60

0      503.016667
1      537.900000
2      564.583333
3      564.600000
4      582.283333
          ...    
103    903.433333
104    909.316667
105    910.500000
106    920.433333
107    940.816667
Name: Time, Length: 108, dtype: float64

In [12]:
df2['Total_Minutes']= df2['Time'].dt.total_seconds()/ 60

In [13]:
df2['Total_Minutes'] = df2['Total_Minutes'].round().astype(int)

In [14]:
df2.head()

,Place,First,Last,City,State,Age,Division,Time,fullname,Total_Minutes
0,1,Daniel,Wilson,Tulsa,OK,35,M,0 days 08:23:01,Daniel Wilson,503
1,2,Eric,Davis,Greenwood,IN,38,M,0 days 08:57:54,Eric Davis,538
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,0 days 09:24:35,Stewart Edwards,565
3,4,Ron,Hammett,Montverde,FL,53,M,0 days 09:24:36,Ron Hammett,565
4,5,Seth,Cain,Geneva,FL,44,M,0 days 09:42:17,Seth Cain,582


In [15]:
df2.rename(columns = {'Division':'Gender'},inplace=True)
df2.head()

,Place,First,Last,City,State,Age,Gender,Time,fullname,Total_Minutes
0,1,Daniel,Wilson,Tulsa,OK,35,M,0 days 08:23:01,Daniel Wilson,503
1,2,Eric,Davis,Greenwood,IN,38,M,0 days 08:57:54,Eric Davis,538
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,0 days 09:24:35,Stewart Edwards,565
3,4,Ron,Hammett,Montverde,FL,53,M,0 days 09:24:36,Ron Hammett,565
4,5,Seth,Cain,Geneva,FL,44,M,0 days 09:42:17,Seth Cain,582


In [16]:
def get_lat_long(city,state):
    address = f"{city},{state}"
    try:
        geolocator=Nominatim(user_agent="running", timeout=10)
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None,None
    except GeocoderTimedOut:
        return None, None

In [19]:
#It looks like you're defining a function get_lat_long that takes a city and state as input
#and returns their latitude and longitude using geocoding.
#This function uses the geopy library, specifically the Nominatim geocoder.
#Here's a breakdown of the function:
#It constructs the address string using the provided city and state.
#It tries to geocode the address using the Nominatim geocoder with a timeout of 10 seconds.
#If the geocoding is successful and a location is found, it returns the latitude and longitude.
#If the geocoding fails due to a timeout, it returns None for both latitude and longitude.
#This function seems well-defined, but you might want to ensure you have the
#necessary imports (Nominatim and GeocoderTimedOut) from the geopy library for it to work correctly.
#Also, remember to handle other possible exceptions if necessary.

In [17]:
df2['latitude'],df2['longitude'] = zip(*df2.apply(lambda x: get_lat_long(x['City'],x['State']),axis=1))

In [ ]:
#The apply function is used to apply a function along an axis of the DataFrame.
#In your case, you're applying a lambda function to each row (axis=1),
#where the lambda function calls get_lat_long with the values from the 'City' and 'State' columns of the DataFrame.

#Then, the zip(*...) function is used to unzip the resulting tuples m
#(latitude, longitude pairs) into separate latitude and longitude columns.

In [18]:
df2.head()

,Place,First,Last,City,State,Age,Gender,Time,fullname,Total_Minutes,latitude,longitude
0,1,Daniel,Wilson,Tulsa,OK,35,M,0 days 08:23:01,Daniel Wilson,503,36.156312,-95.992752
1,2,Eric,Davis,Greenwood,IN,38,M,0 days 08:57:54,Eric Davis,538,39.613699,-86.109543
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,0 days 09:24:35,Stewart Edwards,565,29.025813,-80.927127
3,4,Ron,Hammett,Montverde,FL,53,M,0 days 09:24:36,Ron Hammett,565,28.600277,-81.673964
4,5,Seth,Cain,Geneva,FL,44,M,0 days 09:42:17,Seth Cain,582,28.739716,-81.115062


In [20]:
df2['latlong'] = df2['latitude'].astype(str) + ',' +df2['longitude'].astype('str')
df2.head()

,Place,First,Last,City,State,Age,Gender,Time,fullname,Total_Minutes,latitude,longitude,latlong
0,1,Daniel,Wilson,Tulsa,OK,35,M,0 days 08:23:01,Daniel Wilson,503,36.156312,-95.992752,"36.1563122,-95.9927516"
1,2,Eric,Davis,Greenwood,IN,38,M,0 days 08:57:54,Eric Davis,538,39.613699,-86.109543,"39.6136987,-86.1095429"
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,0 days 09:24:35,Stewart Edwards,565,29.025813,-80.927127,"29.0258132,-80.9271271"
3,4,Ron,Hammett,Montverde,FL,53,M,0 days 09:24:36,Ron Hammett,565,28.600277,-81.673964,"28.6002769,-81.673964"
4,5,Seth,Cain,Geneva,FL,44,M,0 days 09:42:17,Seth Cain,582,28.739716,-81.115062,"28.7397163,-81.1150616"


In [23]:
df2.to_csv('ultracleanedupdata_output.csv', index=False)